**The behavior of regex quantifiers is a common source of woes for the regex apprentices. :)**

Regular expressions are immensely useful for extracting information from any texts by searching for one or more matches of a specific search pattern. 

To find a match, the regular expression engine uses the algorithm:

For every position in the string try to match the pattern at that position.
If there is no match, go to the next position.

In [1]:
import re

***

### The special characters. ^ and $ anchors

- ^A - matches any string that starts with 'A'
- B$ - matches a string that ends with 'B'
- ^AxxxxxxxxB$ - matches a string that starts with 'A' and ends with 'B'

In [20]:
# a list filtering to to find only words start with 'a' and end with 'b'
lst = ['ab', 'abc', 'ba', 'cba', 'acb', 'aab', 'abb', 'acc']
pattern = '^a.*b$'
lst = [x for x in lst if re.match(pattern, x)]
print(lst)

['ab', 'acb', 'aab', 'abb']


***

### The special characters. + * ? and {} quantifiers and OR-operation with | or []

- abc+ - matches a string that strictly has 'ab'  and followed by **one or more** 'c'
- abc* - matches a string that strictly has 'ab' and followed by **zero or more** 'c'
- abc? - matches a string that strictly has 'ab' and followed by **zero or one** 'c'
- abc{3} - matches a string that strictly has 'ab' and followed by **3 letters** 'c'
- abc{3,} - matches a string that strictly has 'ab' and followed by **3 or more letters** 'c'
- abc{3,6} - matches a string that strictly has 'ab' and followed by **3 up to 6 letters** 'c'
- a(bc)+ - matches a string that strictly has 'a' followed by **one or more 'bc' sequences**
- a(bc){2,3} - matches a string that strictly has 'a' followed by **2 up to 3 sequences 'bc'**

- a(b|c) - matches a string that strictly has 'a' followed by b or c (and captures b or c)
- a[bc] - the same as the previous
The major difference here is that the ()-version creates a group that can be backreferenced by '\1' in the match but the []-version cannot do this.

***

### Greedy vs lazy (non-greedy) quantifiers

#### Regex greedy match
Quantifiers *, +, ? and {m,n} are 'greedy' by default that means they match as many characters as possible. In other words, the greedy quantifiers capture the longest match from a given position in the string so that the regex pattern is still satisfied.

For example, the regex 'a+' will match as many 'a' letters as possible in a string 'aaaaa', even though the substrings 'a', 'aa', 'aaa' and 'aaaa' all match the regex 'a+'.

Let's illustrate operator's greediness:
Note how '*' and '?' quantifiers match empty string character!

In [42]:
seq = 'aaabaa aaa'
print(re.findall('a*', seq), 'for "a*"') # The zero-or-more regex 'a*'
print(re.findall('a+', seq), 'for "a+"') # The one-or-more regex 'a+'
print(re.findall('a?', seq), 'for "a?"') # The zero-or-one regex 'a?'
print(re.findall('a{3}', seq), 'for "a{3}"') # The repeating regex 'a{3}'
print(re.findall('a{2,3}', seq), 'for "a{2,3}"') # The repeating regex 'a{2,3}'


['aaa', '', 'aa', '', 'aaa', ''] for "a*"
['aaa', 'aa', 'aaa'] for "a+"
['a', 'a', 'a', '', 'a', 'a', '', 'a', 'a', 'a', ''] for "a?"
['aaa', 'aaa'] for "a{3}"
['aaa', 'aa', 'aaa'] for "a{2,3}"


#### Regex lazy (non-greedy, reluctant) match
A lazy match means that the regex engine matches as few characters as possible so that the sequence still can match the pattern in the given string. In other words, the non-greedy quantifier takes the shortest possible match from a given position in the string.

Thus for example,  regex 'a+?' matches the first character 'a' from 'aaa' and is done with it. Then, it moves on to the second character which is also a match and so on.

Non-greedy quantifiers can be produced by appending a question mark symbol '?' to them: '*?', '+?', '??' and '{m,n}?'.

#### Non-greedy zero-or-more operator *?
Note that it matches zero string if possible! Look on the 'bb' segment of string below. Greedy matching takes empty strings at the start and the end of the string only because it greedily consumes the 'bb' substring with the empty 'sub-substring' between 'b' letters, therefore two empty strings are in the result after the first match 'aaa'.
Non-greedy matching treats it another way by collecting all empty substrings including that between 'b' letters so that three empty strings are in the position.   

In [53]:
seq = 'aaabbaa aaa'
print(re.findall('a*', seq), 'is greedy matching')
print(re.findall('a*?', seq), 'is lazy matching')

['aaa', '', '', 'aa', '', 'aaa', ''] is greedy matching
['', 'a', '', 'a', '', 'a', '', '', '', 'a', '', 'a', '', '', 'a', '', 'a', '', 'a', ''] is lazy matching


#### Non-greedy one-or-more operator +?


In [54]:
seq = 'aaabbaa aaa'
print(re.findall('a+', seq), 'is greedy matching')
print(re.findall('a+?', seq), 'is lazy matching')

['aaa', 'aa', 'aaa'] is greedy matching
['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a'] is lazy matching


#### Non-greedy zero-or-one operator ??

In [55]:
seq = 'aaabbaa aaa'
print(re.findall('a?', seq), 'is greedy matching')
print(re.findall('a??', seq), 'is lazy matching')

['a', 'a', 'a', '', '', 'a', 'a', '', 'a', 'a', 'a', ''] is greedy matching
['', 'a', '', 'a', '', 'a', '', '', '', 'a', '', 'a', '', '', 'a', '', 'a', '', 'a', ''] is lazy matching


***

#### Docile behavior of greedy quantifiers: Roll it back when needed.
Let us see to an example below:

In [3]:
sequence = 'Regex is tricky'
pattern = '.*tricky'
re.search(pattern, sequence)

<re.Match object; span=(0, 15), match='Regex is tricky'>

The matching result seems clear enough but there is an important thing how the regex engine really works.

The expression '.*' starts out by greedily matching every single character and captures all the string. Then the engine advances to the next part of the expression - 't' but it fails to match as there are no characters left in the string.  

As result the engine backtracks into the '.\*' again and it gives up the last left letter in 'tricky' - 'y'. The engine again advances to the 't', but 't' fails to match 'y'. The engine again backtracks into the '.\*', which gives up 'k'. The process repeats itself until the '.*' has given up 't'. In the stage the characters t, r, i, c, k and y become all able to match and the overall match will be successful.

So if the quantified part of a pattern has matched so many characters that the rest of the pattern can not match, the engine backtracks to it and make it give up characters it matched earlier. It can be one character or a chunk at a time, that depending on whether the quantifier applies to a single character or to a subpattern like a group e.g. that can match chunks of several characters. After giving up each character or chunk, the engine tries once again to match the rest of the pattern. 

For additional example. The result of matching by patterns 'a+' , '(a+).' and '(a+)..' are the same for the string 'aaa' but the engine works different ways - it backtracks and give up one 'a' and two 'a' as result of last two patterns implementing.

***

#### Helpful behavior of lazy quantifiers: Expand it when needed
Let us see to an example below:

In [9]:
seq = 'Regex is tricky'
pattern = '.*?tricky'
re.search(pattern, seq)

<re.Match object; span=(0, 15), match='Regex is tricky'>

In the instance abobe the subpattern '.\*?' matches a zero character that is the minimum allowed by the '\*' quantifier. Then the engine advances in the pattern and tries to match the 't' character against the 'R' in the sequence. That fails, so the engine backtracks to the '.\*?' subpattern, which expands to match the 'R'. The engine advances both in the pattern and in the string and tries to match the 't' character against the 'e' in the sequence. Once again, the engine has to backtrack. And again. The matching repeats itself until the '.*?' has expanded to match 'Regex is '. At that stage, the following character 't' and all the characters that follow are able to match. The match attempt succeeds. 

As this example showed, how lazy quantifiers expand their match one step at a time in order to match only as much as needed, they cause the engine to backtrack at each step.

To fully grasp how non-greedy quantifiers work, let's look at one more example. The quantified token 'A\*?' matches zero or more 'A' characters few as possible, expanding as needed. Against the string 'AA', depending on the overall pattern, 'A*?' could end up matching no characters at all, 'A' or 'AA':

- ^(A\*?)AA$ - 'A\*?' (captured to Group 1) **matches no characters**. After the anchor '^' asserts that the current position is the beginning of the string, 'A\*?' tries to match the least number of characters allowed by '*', which is zero characters. The engine moves to the next token: the 'A', which matches the first 'A' in 'AA'. The next token matches the second 'A'. The match attempt succeeds and Group 1 contains only zero character.

- ^(A\*?)A$ - 'A*?' (captured to Group 1) **matches one A**. Initially, the 'A*?' matches the zero character at the start of th string. The next subpattern 'A' matches the first 'A' in 'AA'. The engine advances to the next subpattern, but the anchor $ fails to match against the second A. The engine sees that the 'A*?' can be expanded, backtracks and gives up the 'A', which the 'A*?' now expands to match. The engine moves again to the next subpattern: the 'A' matches the second 'A' in the string. The '$' anchor now succeeds. Group 1 contains one 'A'.

- ^A*?$ - 'A*?' **matches all the string 'AA'**. After the 'A*?' matches zero characters, the '$' fails to match. The engine backtracks and allows the 'A*?' to match one 'A'.
Once again, the '$' fails to match because there is one 'A' left in the string. The engine backtracks again and allows the 'A*?' to expand to match the second 'A'. This time, the '$' anchor is able to match. Group 1 contains 'AA'.

***

### The special sequences

- '\A' - the equivalent to '^'
- '\Z' - the equivalent to '$'
- '\n' - the newline symbol
- '\t' - the tabular character
- '\w' - matches any alphanumeric word characters that are uppercase and lowercase letters, digits, underscore '\_' character, a shorthand to [^a-zA-Z0-9_] in  ASCII.
- '\W' - the opposite for '\w'. It matches any non-word character and is equivalent to [^a-zA-Z0-9_] in  ASCII.
- '\d' - matches any Unicode decimal digit (that is, any character in Unicode character category [Nd]). In the ASCII flag is used only [0-9].
- '\D' - matches any character which is not a decimal digit. This is the opposite of \d. The equivalent of [^0-9].
- '\s' - matches Unicode whitespace characters [ \t\n\r\f\v] and also some other characters. If the ASCII flag is used, only [ \t\n\r\f\v].
- '\S' - matches any character which is not a whitespace character. If the ASCII flag is used, the equivalent of [^ \t\n\r\f\v].
- '\b' - matches the empty string, but only at the beginning or end of a word. A word is defined as a sequence of word characters. Formally, \b is defined as the boundary between \w and \W character (or vice versa), or between \w and the beginning/end of the string. This means that r'\bfoo\b' matches 'foo', 'foo.', '(foo)', 'bar foo baz' but not 'foobar' or 'foo3'.
- \B' - matches the empty string, but only when it is not at the beginning or end of a word. This means that r'py\B' matches 'python', 'py3', 'py2', but not 'py', 'py.', or 'py!'. \B is just the opposite of \b, so word characters in Unicode patterns are Unicode alphanumerics or the underscore, although this can be changed by using the ASCII flag. Word boundaries are determined by the current locale if the LOCALE flag is used.

In [3]:
# boundaries
seq_1 = 'abc abc abc aabcc aabcc'
pattern_1 = r'\babc\b' # performs a "whole words only" search
pattern_2 = r'\Babc\B' # matches only if the pattern is fully surrounded by word characters
print(re.findall(pattern_1, seq_1))
print(re.findall(pattern_2, seq_1))

['abc', 'abc', 'abc']
['abc', 'abc']


### Character classes

Characters contained in square brackets [] represent a character class — an enumerated set of characters to match from. A character class matches any single character contained in the class.

A character class can also contain a range of characters separated by a hyphen (-), in which case it matches any single character within the range [a-z] e.g. including boundaries.

In [2]:
seq = 'abc abd abe abf'
pattern = 'ab[cdef]'
re.findall(pattern, seq)

['abc', 'abd', 'abe', 'abf']

In [3]:
seq = 'A9_$88-a45+q'
pattern = '[a-z]'
re.search(pattern, seq)

<re.Match object; span=(7, 8), match='a'>

In [4]:
seq = 'abcabc4abc44abc'
pattern = '[0-9][0-9]'
re.search(pattern, seq)

<re.Match object; span=(10, 12), match='44'>

In [5]:
seq = '111@$abc111'
pattern = '[^0-9]'
re.search(pattern, seq)

<re.Match object; span=(3, 4), match='@'>

In [6]:
seq = 'abc-abc'
pattern = '[0-9\-]'
re.search(pattern, seq)

<re.Match object; span=(3, 4), match='-'>

***

### Grouping and capturing with () and group backreferences

Grouping constructs break up a regex into subexpressions or groups. This may serve two purposes:
- Grouping: A group represents a single syntactic entity. Additional metacharacters apply to the entire group as a unit.
- Capturing: Some grouping constructs also capture the portion of the search string that matches the subexpression in the group. It is possible to retrieve captured matches later through several different mechanisms.

In [12]:
seq = 'ab abc dabca abcabc abccba abcabcd'
pattern = '(abcabc)+'
re.findall(pattern, seq)

['abcabc', 'abcabc']

### Recipies

In [6]:
# not completed, for further 3 or 5

seq = 'aaabbaaaaabbaaaaabbbaaabaaa'
pattern = r'a{3}(?:a{2})?'
re.findall(pattern, seq)

['aaa', 'aaaaa', 'aaaaa', 'aaa', 'aaa']